<a href="https://colab.research.google.com/github/jienn/GAN_practice/blob/master/2_01_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2.3.1 data_load**

In [1]:
import numpy as np
from keras.utils import to_categorical
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
(x_train, y_train),(x_test,y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
NUM_CLASS=10

x_train=x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0


픽셀 값은 0부터 255 사이의 값을 가진다. 신경망 훈련을 위해 0과 1 사이의 실수로 표현해주었다.

In [0]:
y_train = to_categorical(y_train,NUM_CLASS)
y_test= to_categorical(y_test,NUM_CLASS)

In [5]:
x_train[54,1,13,1]

0.32941177

함수 이해 완료!
x_train과 x_test는 신경망훈련을 위해 값 조정한다
y_train은 왜 binary class matrix로 변환하는걸까... 어따 쓰려고?

그거 말고는 데이터 적재 이해완료!

#**2.3.2 모델 만들기**

In [0]:
#!pip show tensorflow

In [0]:
#함수형API를 사용한 네트워크
from keras.layers import Input,Flatten,Dense
from keras.models import Model

In [8]:
input_layer = Input(shape=(32,32,3))

In [0]:
x = Flatten()(input_layer)

In [10]:
x = Dense(units=200,activation='relu')(x)
x = Dense(units=150, activation='relu')(x)

In [11]:
print(x)

Tensor("dense_2/Relu:0", shape=(?, 150), dtype=float32)


In [0]:
output_layer= Dense(units=10, activation='softmax')(x)

model = Model(input_layer, output_layer)

In [13]:
output_layer

<tf.Tensor 'dense_3/Softmax:0' shape=(?, 10) dtype=float32>

dense_4 >> dense_7

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               614600    
_________________________________________________________________
dense_2 (Dense)              (None, 150)               30150     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1510      
Total params: 646,260
Trainable params: 646,260
Non-trainable params: 0
_________________________________________________________________


#**2.3.3 모델컴파일**
  모델에 손실함수와 옵티마이저를 연결한다.

In [0]:
from keras.optimizers import Adam

In [16]:
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.compile()

TypeError: ignored

#**2.3.4 모델 훈련**
아직 모델에게 어떠한 데이터도 전달하지 않았다. 네트워크의 구조를 정의하고 모델에 손실함수와 옵티마이저를 연결했다.

모델을 훈련하려면 다음처럼 fit 메서드를 호출한다.

In [0]:
model.fit(x_train,
          y_train
          , batch_size=32
          ,epochs=10
          , shuffle=True
          )

#**2.3.5 모델 평가**

In [0]:
model.evaluate(x_test,y_test)

In [0]:
CLASSES = np.array(['airplane','automobile','bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck'])

preds = model.predict(x_test)
preds_single = CLASSES[np.argmax(preds, axis= -1)]
actual_single = CLASSES[np.argmax(y_test, axis = -1)]

In [0]:
#print(preds[8])

샘플마다 10개의 클래스 확률을 담은 벡터가 preds에 반환됨.

In [0]:
np.argmax(preds[8], axis=-1)

각 샘플에서 최대 값을 반환하는데, 곱하기 10한 상태...
0.383483 -> 4

In [0]:
import matplotlib.pyplot as plt

n_to_show = 10
indices = np.random.choice(range(len(x_test)), n_to_show)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i, idx in enumerate(indices):
    img = x_test[idx]
    ax = fig.add_subplot(1, n_to_show, i+1)
    ax.axis('off')
    ax.text(0.5, -0.35, 'pred = ' + str(preds_single[idx]), fontsize=10, ha='center', transform=ax.transAxes) 
    ax.text(0.5, -0.7, 'act = ' + str(actual_single[idx]), fontsize=10, ha='center', transform=ax.transAxes)
    ax.imshow(img)

#**2.4 모델 성능 향상**
##**2.4.1 합성곱 층**